# Database Admin 101 - Lab

## Introduction 

In this lab you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Create a SQL database
* Create a SQL table
* Create rows in a SQL table
* Alter entries in a SQL table
* Delete entries in a SQL table
* Commit changes via sqlite3


## The Scenario

You are looking to design a database for a school which will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [2]:
# Import sqlite3 and create the database
import sqlite3
conn = sqlite3.connect('schoo.sqlite')
cur = conn.cursor()

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), email, telephone number, street, city, state and zipcode. Be sure to also create a primary key for the table. 

In [4]:
# Create contactInfo table
cur.execute("""CREATE TABLE contactInfo (
                                         id INTEGER PRIMARY KEY,
                                         firstName TEXT,
                                         lastName TEXT,
                                         role TEXT,
                                         email TEXT,
                                         phoneNumber TEXT,
                                         street TEXT,
                                         city TEXT,
                                         state TEXT,
                                         zipcode INTEGER)""")

## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [5]:
# Code to load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [6]:
contacts

[{'firstName': 'Christine',
  'lastName': 'Holden',
  'role': 'staff',
  'telephone ': 2035687697,
  'street': '1672 Whitman Court',
  'city': 'Stamford',
  'state': 'CT',
  'zipcode ': '06995'},
 {'firstName': 'Christopher',
  'lastName': 'Warren',
  'role': 'student',
  'telephone ': 2175150957,
  'street': '1935 University Hill Road',
  'city': 'Champaign',
  'state': 'IL',
  'zipcode ': '61938'},
 {'firstName': 'Linda',
  'lastName': 'Jacobson',
  'role': 'staff',
  'telephone ': 4049446441,
  'street': '479 Musgrave Street',
  'city': 'Atlanta',
  'state': 'GA',
  'zipcode ': '30303'},
 {'firstName': 'Andrew',
  'lastName': 'Stepp',
  'role': 'student',
  'telephone ': 7866419252,
  'street': '2981 Lamberts Branch Road',
  'city': 'Hialeah',
  'state': 'Fl',
  'zipcode ': '33012'},
 {'firstName': 'Jane',
  'lastName': 'Evans',
  'role': 'student',
  'telephone ': 3259909290,
  'street': '1461 Briarhill Lane',
  'city': 'Abilene',
  'state': 'TX',
  'zipcode ': '79602'},
 {'firstNa

In [15]:
# Iterate over the contact list and populate the contactInfo table
for contact in contacts:
    firstName = contact['firstName']
    lastName = contact['lastName']
    role = contact['role']
    telephone = contact['telephone ']
    street = contact['street']
    city = contact['city']
    state = contact['state']
    zipcode = contact['zipcode ']
    cur.execute("""INSERT INTO contactInfo (firstName, lastName, role, phoneNumber, street, city, state, zipcode)
                   VALUES ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');""".format(firstName, lastName, role, telephone, street, city, state, zipcode)) 
                        


**Query the Table to Ensure it is populated**

In [17]:
# Query the table to ensure it is populated
cur.execute("""SELECT * FROM contactInfo LIMIT 2;""").fetchall()

[(1,
  'Christine',
  'Holden',
  'staff',
  None,
  '2035687697',
  '1672 Whitman Court',
  'Stamford',
  'CT',
  6995),
 (2,
  'Christopher',
  'Warren',
  'student',
  None,
  '2175150957',
  '1935 University Hill Road',
  'Champaign',
  'IL',
  61938)]

## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [18]:
# Commit changes
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [22]:
# Create the grades table
cur.execute("""CREATE TABLE grades (
                                    userId INTEGER NOT NULL,
                                    courseId INTEGER NOT NULL,
                                    grade TEXT,
                                    PRIMARY KEY(userId, courseId));""")

## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [37]:
# Find the duplicate entry
cur.execute("""SELECT firstName, lastName, phoneNumber, street, COUNT(*)
               FROM contactInfo
               GROUP BY 1, 2, 3, 4
               HAVING COUNT(*) > 1;""").fetchall()

[('Jane', 'Evans', '3259909290', '1461 Briarhill Lane', 2)]

In [35]:
# Find the indices the duplicate entry
cur.execute("""SELECT id 
               FROM contactInfo 
               WHERE (firstName = 'Jane') & (lastName = 'Evans');""").fetchall()

[(5,), (6,)]

In [38]:
# Delete one of the duplicates
cur.execute("""DELETE FROM contactInfo WHERE id = 5;""")

In [40]:
# Check that the duplicate entry was removed.
cur.execute("""SELECT *
               FROM contactInfo
               WHERE (firstName = 'Jane') & (lastName = 'Evans');""").fetchall()

[(6,
  'Jane',
  'Evans',
  'student',
  None,
  '3259909290',
  '1461 Briarhill Lane',
  'Abilene',
  'TX',
  79602)]

## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [43]:
# Update Ed's address
cur.execute("""UPDATE contactInfo
               SET street = '2910 Simpson Avenue',
                   city = 'York',
                   state = 'PA',
                   zipcode = 17403
               WHERE (firstName = 'Ed') & (lastName = 'Lyman');""")

In [44]:
# Query the database to ensure the change was made
cur.execute("""SELECT * 
               FROM contactInfo 
               WHERE (firstName = 'Ed') & (lastName = 'Lyman');""").fetchall()

[(8,
  'Ed',
  'Lyman',
  'student',
  None,
  '5179695576',
  '2910 Simpson Avenue',
  'York',
  'PA',
  17403)]

## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [45]:
# Commit changes
conn.commit()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 